# Analysis

## Calculate correlation

In [2]:
import scipy
import numpy as np

In [3]:
bnum = 8
corr_all_Tsamples = []
secret_range = 2**(bnum)
M = round(secret_range / 2)
#hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]

In [4]:
mult_result = (256 * 10) % secret_range
#print(hw[mult_result])

0


In [7]:
# Ulity functions
def plot_mult(corr_list, _type = "norm", mult=True, xlim:list[int]=None, ylim:list[int]=None, graph_title="Multiple Correlation Traces", ylabel="Correlation"):
  if not mult: #len(corr_list.shape) >  1:
    corr_list = [corr_list]
  image = plt.figure()
  plt.title(graph_title)
  plt.xlabel("Index")
  plt.ylabel(ylabel)
  if xlim is not None:
      plt.xlim(xlim)
  if ylim is not None:
      plt.ylim(ylim)
  for corr in corr_list:
      plt.plot(corr)#, label=f'Trace {i+1}')
  plt.show()
  os.makedirs('./figures', exist_ok=True)
  image.savefig(f'./figures/{_type}_corr.png')
  plt.close()  # Close the figure to free memory
def createDiffWave(waves, name):
        diff_waves = []
        square_waves = []
        avg_wave = np.mean(waves,axis=0)
        
        for i in waves:
            diff_waves.append(np.subtract(i,avg_wave))
        for i in diff_waves:
            square_wave = []
            for ii in i:
                square_wave.append(ii**2)
            square_waves.append(square_wave)
        
        diff_avg = np.mean(diff_waves,axis=0)
        var = np.mean(square_waves,axis=0)
    
        plot_mult(np.array(waves[0]), _type = f"wave0_{name}", mult=False)
        plot_mult(np.array(avg_wave), _type = f"avg_wave_{name}", mult=False)
        plot_mult(np.array(diff_avg), _type = f"difference_wave_{name}", mult=False)
        plot_mult(np.array(var), _type = f"variation_{name}", mult=False)
        return diff_avg, avg_wave, var, waves
def get_overflowed_val(val = 0, bnum = 8):
    border = 2**(bnum-1)
    if val >= border:
        val = val % (border*2)
        if val >= border:
            val = -border + (val - border)
    return val
def hamming_weight(x, is_int = True):
    if not is_int:
        return np.count_nonzero(x == 1)
    return bin(x).count("1")
def calc_p_val(corr: float, set_len: int, mode: int = 0) -> float:
  t_stat = corr*( ((set_len-2) / (1 - corr**2))**0.5 )
  if mode == 0:
    df = set_len - 2  # degrees of freedom
    p_value = 2 * stats.t.sf(abs(t_stat), df)  # Two-tailed p-value
    return p_value
def calc_intermediate_val(secret_value: int, knonw_input: int):
    #return hamming_weight(np.uint32(np.uint8(secret_value) * np.uint8(knonw_input)))
    return hamming_weight(np.uint32(secret_value) * np.uint32(knonw_input))
def calc_intermediate_val2(secret_value: int, knonw_input: int):
    #return hamming_weight(np.uint32(np.uint8(secret_value) * np.uint8(knonw_input)) % 256)
    return hamming_weight(np.uint32(secret_value) * np.uint32(knonw_input) % 256)
def calc_intermediate_val3(secret_value: int, knonw_input: int):
    #return hamming_weight(np.uint32(np.uint8(secret_value) * np.uint8(knonw_input)) % 256)
    return np.uint32(secret_value) * np.uint32(knonw_input)

In [17]:
hamming_weight(np.uint32(1023 * 1))

10

In [ ]:
hamming_weight(np.uint8(255))

In [36]:
hamming_weight(np.uint8(255))

8

In [35]:
255 * 110

28050

In [18]:
calc_intermediate_val2(1023, 1)

8

In [14]:
calc_intermediate_val(1023, 1)

10

In [12]:
calc_intermediate_val(255, 240)

8

In [27]:
# Agregate Functions: For calculation of correlation above, waves dataset, with hypotetical created traces based of known_input
def ancientV2_ANN_CPA_OTS(known_input : list[int], waves = np.array([]), n_traces: int = None, trace_len: int = None, ith_weight: int = 0, calc_p_value: bool = False): # Vanilla ANN CPA, for one time_sample a
    # Verification of parameters:
    if n_traces is None:
      n_traces = len(known_input)
    if trace_len is None:
      trace_len = len(waves[0])

    # Preparational calculations
    bnum = 8
    corr_all_Tsamples = []
    secret_range = 2**(bnum)
    #hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]

    # Calculating statistics for traces in waves set
    qsum_L_list = []
    l_diff_list = []
    for time_sample in range(trace_len):
      # Calculation preparations: target sets extraction
      L = waves[:,time_sample]
      L_mean = np.mean(L)
      # Calculation
      l_diff = L - L_mean  # Vectorized
      qsum_L = np.sum(np.square(l_diff))  # Sum of squared differences
      # Tidy up
      qsum_L_list.append(qsum_L)
      l_diff_list.append(l_diff)

    for secret_value in trange(secret_range, desc='Calculating Correlations for the Secret-Key: '): # For current WeightHypothesis do
        corr_Tsamples = []
        #H = [calc_intermediate_val3(secret_value, known_input[j]) for j in range(n_traces)]
        H = np.uint32(known_input *  secret_value)
        H_mean = np.mean(H)
        h_diff = np.array(H) - H_mean
        qsum_H = np.sum(np.square(h_diff))

        # For current WeightHypothesis, create an Correlation, this vector needs to be created for each time sample:
        for time_sample in range(trace_len):
            # Calculation preparations: target sets extraction
            L = waves[:,time_sample]
            l_diff = l_diff_list[time_sample]
            sum_HL = np.sum(h_diff * l_diff)  # Dot product
            # Calculation
            divider = (qsum_H ** 0.5) * (qsum_L_list[time_sample] ** 0.5)
            corr = sum_HL / divider if divider != 0 else 0
            if calc_p_value:
              t_stat = corr*( ((n_traces-2) / (1 - corr**2))**0.5 )
              df = n_traces - 2  # degrees of freedom
              p_value = 2 * stats.t.sf(abs(t_stat), df)  # Two-tailed p-value
              corr = [corr, p_value]
            # Tidy up
            corr_Tsamples.append(corr)
        corr_all_Tsamples.append(corr_Tsamples)
    return np.array(corr_all_Tsamples)

In [33]:
temp = [256,258,4550,1666,2245,6546,7888,6788]
tmp_a = np.array(temp)

a = np.uint32(tmp_a - 1)
a = np.array([bin(i).count("1") for i in a])
print(a)
print(temp)
HH = np.uint16(tmp_a *  2)
print(HH)
print(np.array(temp))
for i in np.array(temp):
    print(i)

[ 8  2  6  4  4  6 10  6]
[256, 258, 4550, 1666, 2245, 6546, 7888, 6788]
[  512   516  9100  3332  4490 13092 15776 13576]
[ 256  258 4550 1666 2245 6546 7888 6788]
256
258
4550
1666
2245
6546
7888
6788


In [34]:
bnum = 16
secret_range = 2**(bnum)
hw = [hamming_weight(secret_value) for secret_value in range(secret_range)] 

In [40]:
# Agregate Functions: For calculation of correlation above, waves dataset, with hypotetical created traces based of known_input
def oldV2_ANN_CPA_OTS(known_input : list[int], waves = np.array([]), n_traces: int = None, trace_len: int = None, ith_weight: int = 0, calc_p_value: bool = False): # Vanilla ANN CPA, for one time_sample a
    # Verification of parameters:
    if n_traces is None:
      n_traces = len(known_input)
    if trace_len is None:
      trace_len = len(waves[0])

    # Preparational calculations
    bnum = 8
    corr_all_Tsamples = []
    secret_range = 2**(bnum)
    global hw

    # Calculating statistics for traces in waves set
    qsum_L_list = []
    l_diff_list = []
    for time_sample in range(trace_len):
      # Calculation preparations: target sets extraction
      L = waves[:,time_sample]
      L_mean = np.mean(L)
      # Calculation
      l_diff = L - L_mean  # Vectorized
      qsum_L = np.sum(np.square(l_diff))  # Sum of squared differences
      # Tidy up
      qsum_L_list.append(qsum_L)
      l_diff_list.append(l_diff)

    for secret_value in trange(secret_range, desc='Calculating Correlations for the Secret-Key: '): # For current WeightHypothesis do
        corr_Tsamples = []
        H = [hw[np.uint32(known_input[i] *  secret_value) % 256] for i in range(n_traces)]
        H_mean = np.mean(H)
        h_diff = H - H_mean
        qsum_H = np.sum(np.square(h_diff))

        # For current WeightHypothesis, create an Correlation, this vector needs to be created for each time sample:
        for time_sample in range(trace_len):
            # Calculation preparations: target sets extraction
            L = waves[:,time_sample]
            l_diff = l_diff_list[time_sample]
            sum_HL = np.sum(h_diff * l_diff)  # Dot product
            # Calculation
            divider = (qsum_H ** 0.5) * (qsum_L_list[time_sample] ** 0.5)
            corr = sum_HL / divider if divider != 0 else 0
            if calc_p_value:
              t_stat = corr*( ((n_traces-2) / (1 - corr**2))**0.5 )
              df = n_traces - 2  # degrees of freedom
              p_value = 2 * stats.t.sf(abs(t_stat), df)  # Two-tailed p-value
              corr = [corr, p_value]
            # Tidy up
            corr_Tsamples.append(corr)
        corr_all_Tsamples.append(corr_Tsamples)
    return np.array(corr_all_Tsamples)

In [38]:
# Agregate Functions: For calculation of correlation above, waves dataset, with hypotetical created traces based of known_input
def V2_ANN_CPA_OTS(known_input : list[int], waves = np.array([]), n_traces: int = None, trace_len: int = None, ith_weight: int = 0, calc_p_value: bool = False): # Vanilla ANN CPA, for one time_sample a
    # Verification of parameters:
    if n_traces is None:
      n_traces = len(known_input)
    if trace_len is None:
      trace_len = len(waves[0])

    # Preparational calculations
    bnum = 8
    corr_all_Tsamples = []
    secret_range = 2**(bnum)
    global hw

    # Calculating statistics for traces in waves set
    qsum_L_list = []
    l_diff_list = []
    for time_sample in range(trace_len):
      # Calculation preparations: target sets extraction
      L = waves[:,time_sample]
      L_mean = np.mean(L)
      # Calculation
      l_diff = L - L_mean  # Vectorized
      qsum_L = np.sum(np.square(l_diff))  # Sum of squared differences
      # Tidy up
      qsum_L_list.append(qsum_L)
      l_diff_list.append(l_diff)

    for secret_value in trange(secret_range, desc='Calculating Correlations for the Secret-Key: '): # For current WeightHypothesis do
        corr_Tsamples = []
        H = [hw[np.uint32(known_input[i] *  secret_value)] for i in range(n_traces)]
        H_mean = np.mean(H)
        h_diff = H - H_mean
        qsum_H = np.sum(np.square(h_diff))

        # For current WeightHypothesis, create an Correlation, this vector needs to be created for each time sample:
        for time_sample in range(trace_len):
            # Calculation preparations: target sets extraction
            L = waves[:,time_sample]
            l_diff = l_diff_list[time_sample]
            sum_HL = np.sum(h_diff * l_diff)  # Dot product
            # Calculation
            divider = (qsum_H ** 0.5) * (qsum_L_list[time_sample] ** 0.5)
            corr = sum_HL / divider if divider != 0 else 0
            if calc_p_value:
              t_stat = corr*( ((n_traces-2) / (1 - corr**2))**0.5 )
              df = n_traces - 2  # degrees of freedom
              p_value = 2 * stats.t.sf(abs(t_stat), df)  # Two-tailed p-value
              corr = [corr, p_value]
            # Tidy up
            corr_Tsamples.append(corr)
        corr_all_Tsamples.append(corr_Tsamples)
    return np.array(corr_all_Tsamples)

In [7]:
from typing import Callable as function
def corr_2TraceMatrixes(traces: np.ndarray, inputs: list[int],
                        trace_len: int = 24000, n_traces: int = 1000,
                        xth_secret_val: int = 0, secret_range: int = 256,
                        corr_func: function = np.corrcoef): # function can be also sc.pearsonr
  #hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]
  leak_model = np.array([[calc_intermediate_val(secret_val, inputs[j]) for j in range(n_traces)] for secret_val in range(secret_range)])
  corr_all = []
  for i in trange(secret_range, desc='Calculating Correlations for secret key'):
    corr_curr = []
    to_corr_array = leak_model[i, :]
    for j in range(trace_len):
      corr_curr.append( corr_func(traces[:,j], to_corr_array))
    corr_all.append(corr_curr)
  return np.array(corr_all)
from typing import Callable as function
import scipy.stats as sc
def sc_pearson_2TraceMatrixes(traces: np.ndarray, inputs: list[int],
                        trace_len: int = 24000, n_traces: int = 1000,
                        xth_secret_val: int = 0, secret_range: int = 256): # function can be also sc.pearsonr
  #hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]
  leak_model = np.array([[calc_intermediate_val(secret_val, inputs[j]) for j in range(n_traces)] for secret_val in range(secret_range)])
  corr_all = []
  for i in trange(secret_range, desc='Calculating Correlations for secret key'):
    corr_curr = []
    #to_corr_array = np.array([leak_model[i,:] for n in range(trace_len)]).transpose()
    #corr_all.append(scipy.stats.pearsonr(traces[:,0:trace_len], to_corr_array, axis=0))
    for j in range(trace_len):
      corr_curr.append(sc.pearsonr(traces[:,j], leak_model[i,:]))
    #corr_all.append(sc.pearsonr(traces[:,0:trace_len], to_corr_array, axis=0))
    corr_all.append(corr_curr)
  return np.array(corr_all)

## Extract weights from correlation map

In [22]:
class TextColor():
        black = '\033[30m'
        red = '\033[31m'
        green = '\033[32m'
        orange = '\033[33m'
        blue = '\033[34m'
        purple = '\033[35m'
        cyan = '\033[36m'
        lightgrey = '\033[37m'
        darkgrey = '\033[90m'
        lightred = '\033[91m'
        lightgreen = '\033[92m'
        yellow = '\033[93m'
        lightblue = '\033[94m'
        pink = '\033[95m'
        lightcyan = '\033[96m'
        reset = '\033[0m'

In [7]:
def show_extraction_method_info(vals_locMaxVector: list[int], indx_ColmnMax: list[int], method_name: str = f'xth-level degree', color: str = "blue"):
    fig = plt.figure()
    plt.plot(vals_locMaxVector, color=color)
    plt.title(f'{method_name} Local maxims graph')
    plt.xlabel("Time Sample")
    plt.ylabel("Secret Value")
    plt.show()
    plt.close()

    
    indx_localMax = []
    for i in range(len(vals_locMaxVector)):
        val_toAppend = 0
        if vals_locMaxVector[i] > 0:
            val_toAppend = indx_ColmnMax[i]
        indx_localMax.append(val_toAppend)
    local_maxims = [int(indx_ColmnMax[i]) for i in range(len(indx_ColmnMax)) if vals_locMaxVector[i] > 0]
    uniques_set = [ x for i, x in enumerate(local_maxims) if x not in local_maxims[:i]]
    print(f"Number of uniques: {len(uniques_set)}")
    print(f"Number of local maxims: {len(local_maxims)}")
    print(f"Uniquess: {TextColor.pink}{uniques_set}{TextColor.reset}")
    print(f"Local maxims: {TextColor.orange}{local_maxims}{TextColor.reset}")
    fig = plt.figure()
    plt.plot(indx_localMax, color=color)
    plt.title(f"{method_name} indexes graph")
    plt.xlabel("Time Sample")
    plt.ylabel("Index")
    plt.show()
    plt.close()

In [4]:
def get_weight(correlation_matrix: np.array, min_treshold = 0.0, peak_range = 0.3, show_info=False, lastIndex = 0, map_2dgrMaxs = None, mountain_half_dist = 20, level = 5):
    #FIND index of highest value for each column (correlation of time sample)
    indx_ColmnMax = correlation_matrix.argmax(axis=0) #Return an vector of indexes of max values in each column of matrix
    
    #print(f"Input Vector: {correlation_matrix[1]}")

    #CREATE an array from previous finds
    colmn_MaxVector = np.array([correlation_matrix[indx_ColmnMax[i]][i] for i in range(len(indx_ColmnMax))]) #Return a vector of max values for each column in matrix
    for i in range(len(colmn_MaxVector)):
        if colmn_MaxVector[i] < min_treshold:
            colmn_MaxVector[i] = 0
    #FIND GLOBAL MAX
    indx_globMax = np.argmax(colmn_MaxVector)
    val_globMax = colmn_MaxVector[indx_globMax]
    vec_len = len(colmn_MaxVector)
    #print(f"Max Vector: {colmn_MaxVector}")


    if map_2dgrMaxs is None:
        print(f"{len([i for i in colmn_MaxVector if i != 0])}")
        vals_locMaxVector0 = []
        tmp_indices = []
        for i in range(len(colmn_MaxVector)):
            tmp_indices.append(i)
            vals_locMaxVector0.append(0)
        for i in range(level):
            tmp_array = []
            for indx in tmp_indices:
                tmp_array.append(colmn_MaxVector[indx])
            indices, props = scipy.signal.find_peaks(tmp_array)
            for indx in range(len(indices)):
                indices[indx] = tmp_indices[indices[indx]]
            tmp_indices = indices
        for indx in tmp_indices:
            if val_globMax-peak_range <= colmn_MaxVector[indx]:
                vals_locMaxVector0[indx] = colmn_MaxVector[indx]
        
        vals_locMaxVector1 = []
        #FIND 1. local maxims and zero-out all values in tresshold range to global_correlation
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = colmn_MaxVector[indx]
            val_toAppend = 0
            if (value + peak_range) >= val_globMax and ((indx+1) < vec_len) and (value > colmn_MaxVector[indx+1]):
                val_toAppend = value
            vals_locMaxVector1.append(val_toAppend)
        secret_corr = 0 #FINAL correlation
        secret_colmn = 0 #Column/Time sample of Final correlation
    
        #FIND 2. local maxims
        vals_locMaxVector2 = []
        last_val = vals_locMaxVector1[0]
        last_indx = 0
        zero_counter = 0
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = vals_locMaxVector1[indx]
            vals_locMaxVector2.append(0)
            if value != 0:#Cross the desert, and on start of the next mountain, reminiscence about last mountain and save it to the memoar 
                if zero_counter > mountain_half_dist:
                    vals_locMaxVector2[last_indx] = last_val
                    #Reset
                    last_val = 0
                    last_indx = indx
                else:
                    if last_val < value:
                        last_val = value
                        last_indx = indx
                zero_counter = 0
            else:
                zero_counter += 1
        vals_locMaxVector2[last_indx] = last_val



        #FIND 3. local maxims
        vals_locMaxVector3 = []
        last_val = vals_locMaxVector1[0]
        last_indx = 0
        zero_counter = 0
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = vals_locMaxVector1[indx]
            vals_locMaxVector3.append(0)
            if value > (last_val-(peak_range/2)):#Cross the desert, and on start of the next mountain, reminiscence about last mountain and save it to the memoar 
                if zero_counter > mountain_half_dist:
                    vals_locMaxVector3[last_indx] = last_val
                    #Reset
                    last_val = 0
                    last_indx = indx
                else:
                    if  last_val > value:
                        vals_locMaxVector3[last_indx] = last_val
                    last_val = value
                    last_indx = indx
                zero_counter = 0
            else:
                zero_counter += 1
        vals_locMaxVector3[last_indx] = last_val


        if show_info:    
            print(f" Index of global max: {indx_globMax} and global_max_val: {val_globMax}")
            show_extraction_method_info(vals_locMaxVector1, indx_ColmnMax, method_name = f'1th degree', color = "red")
            show_extraction_method_info(vals_locMaxVector0, indx_ColmnMax, method_name = f'{level}-level', color = "green")
            show_extraction_method_info(vals_locMaxVector2, indx_ColmnMax, method_name = f'2th-var degree', color = "blue")
            show_extraction_method_info(vals_locMaxVector3, indx_ColmnMax, method_name = f'3th-var degree', color = "purple")
    else:
        vals_locMaxVector2 = map_2dgrMaxs
    
    secret_corr = 0 #FINAL correlation
    secret_colmn = 0 #Column/Time sample of Final correlation
    nextIndex = lastIndex
    found = False
    print(f"In range {lastIndex} - {len(indx_ColmnMax)}")
    for indx in range(lastIndex, len(indx_ColmnMax)): #Find first local maxim in the peak_range from global maxim
        value = vals_locMaxVector2[indx]
        if value != 0:
            if not found:
                secret_corr = value
                secret_colmn = indx
                found = True
                continue
            nextIndex =  indx - round((indx - secret_colmn) / 2)
            break
    secret_val = indx_ColmnMax[secret_colmn]

    
    print(f" Found the soonest secret value is {secret_val}, at the time sample {secret_colmn}, with correlation {secret_corr = :.3f}")
    return secret_val, secret_colmn, secret_corr, vals_locMaxVector2, colmn_MaxVector, nextIndex, vals_locMaxVector2
#results_directory = {}
#secret_value, time_sample, correlation, localMax_vector = get_weight(np.array(ncorr_all), show_info=True)

In [7]:
print("✔️ The Analyser succesfuly runned.")

✔️ The Analyser succesfuly runned.
